# NL2SQL Agent: Converts Natural Language to SQL Query

An agent that leverages Large Language Models (LLMs) and the ReAct (Reason+Act) framework to convert natural language queries into SQL and perform various database operations.

## How it works

1. **Natural Language Understanding**: The LLM-powered agent interprets user queries expressed in natural language.

2. **ReAct Framework**: Using the ReAct (Reason+Act) approach, the agent breaks down complex queries into a series of steps, reasoning about the best approach to fulfill the user's request.

3. **Tool Selection**: The agent selects from a suite of specialized database tools to accomplish each step:
   - Listing available tables
   - Retrieving table schemas and sample data
   - Validating SQL queries
   - Executing SQL queries
   - Analyzing table relationships
   - Gathering table statistics

4. **SQL Generation**: Based on the user's intent and the database structure, the agent generates appropriate SQL queries.

5. **Execution and Refinement**: The agent executes the generated SQL, analyzes the results, and refines its approach if necessary.

6. **User-Friendly Output**: Finally, the agent presents the results in a clear, human-readable format.

This approach allows users to interact with databases using natural language, without needing to know SQL or the specific database structure. The ReAct agent's ability to reason about the problem and use the appropriate tools makes it highly flexible and capable of handling a wide range of database-related tasks.

## Database Setup
This project uses the MySQL Sakila sample database. To reproduce the results and use the SQL agent:

Install MySQL if you haven't already.
Sakila database is already downloaded from the official MySQL website and can be found in `nl2sql_tools`
Follow the installation instructions provided with the Sakila database to set it up in your MySQL instance.
Update the database connection settings in the project configuration to point to your Sakila database.

```bash
brew install mysql #install on mac os
brew services start mysql 
mysql_secure_installation # setup password
mysql -u root -p # connect to db
```



### Imports

In [1]:
import os
from llama_index.core.agent import ReActAgent 
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from dotenv import load_dotenv
import nest_asyncio

from nl2sql.tools import (
    list_tables,
    get_table_schema_and_sample,
    validate_sql_query,
    run_sql_query,
    get_table_relationships,
    get_table_statistics,
)

from nl2sql import initialize_db_connection

nest_asyncio.apply()

### Select your LLM and Initialize the DB

In [2]:
load_dotenv()

OPEN_AI_MODEL_NAME = "gpt-4o-2024-08-06" # "gpt-4o-mini-2024-07-18" # 
KEY = os.environ.get("OPENAI_API_KEY")

llm = OpenAI(model=OPEN_AI_MODEL_NAME,api_key=KEY)
initialize_db_connection('localhost', 'root', os.environ.get("DB_Password"), 'sakila')

Database connection successful


### DB Tools Functions

- `list_tables()`: Retrieves a list of all tables in the connected database.

- `get_table_schema_and_sample(table_name: str)`: Returns the schema and a sample of 3 rows from the specified table.

- `validate_sql_query(query: str)`: Checks if a given SQL query is valid without executing it.

- `run_sql_query(query: str)`: Executes the provided SQL query and returns the results as formatted text.

- `get_table_relationships()`: Retrieves and formats information about foreign key relationships between tables in the database.

- `get_table_statistics(table_name: str)`: Provides statistics for a given table, including row count, column details, and index information.


In [3]:
list_tables_tool = FunctionTool.from_defaults(fn=list_tables)
get_table_schema_and_sample_tool = FunctionTool.from_defaults(fn=get_table_schema_and_sample)


get_table_relationships_tool = FunctionTool.from_defaults(fn=get_table_relationships)
get_table_statistics_tool = FunctionTool.from_defaults(fn=get_table_statistics)

validate_sql_query_tool = FunctionTool.from_defaults(fn=validate_sql_query)
run_sql_query_tool = FunctionTool.from_defaults(fn=run_sql_query)

tools = [list_tables_tool, get_table_schema_and_sample_tool, 
         get_table_relationships_tool, get_table_statistics_tool,
         validate_sql_query_tool, run_sql_query_tool]

### Prompt for your Agent

In [4]:
# Option 1: Initialize OpenAIAgent
context = """You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.

        Your task is to construct, execute sql and return the summary of the result based on the natural language queries of the users.
        
        Instruction:
        1. Check if there is need to rephrase or elaborate natural language query from the user.
        2. For complex query break down the query in small parts and then combine them together to solve the main task.
        3. Limit the result to 20 rows.
        4. Generated new query if syntax is invalid.
        5. Validate the result and check if it is answering the user natural langauge query, if not regenerate the sql query.
        """

agent = ReActAgent.from_tools(tools, llm=llm, verbose=True,context=context,max_iterations=20)

prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")


Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}

Here is some context to help you answer the question and plan:
{context}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a 

### Example 1: **Find the names of customers who rented more than 5 films in the month of May 2005.**

In [9]:
nl_query_1 = "Find the names of customers who rented more than 5 films in the month of May 2005"

# Expected
"""
SELECT c.first_name, c.last_name, COUNT(r.rental_id) AS rental_count
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
WHERE MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005
GROUP BY c.customer_id
HAVING rental_count > 5;
"""
# Generated
"""
SELECT c.first_name, c.last_name
    FROM customer c
    JOIN rental r ON c.customer_id = r.customer_id
    WHERE MONTH(r.rental_date) = 5 AND YEAR(r.rental_date) = 2005
    GROUP BY c.customer_id
    HAVING COUNT(r.rental_id) > 5
"""

response = agent.chat(nl_query_1)

> Running step 9a5b428b-930a-4b9f-a084-e12f1eadd57a. Step input: Find the names of customers who rented more than 5 films in the month of May 2005
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step ae4385c3-8eef-4b2f-90f7-cbc98fc35caf. Step input: None
Thought: The current language of the user is English. I need to use a tool to help me answer the question. First, I will list all available tables to understand the database structure.
Action: list_tables
Action Input: {}
Observation: ['actor', 'actor_info', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'nicer_but_slower_film_list', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store']
> Running step 344cb65e-ed5b-433c-a028-3dea38ce7a40. Step input: None
Thought: To find the names of customers who rented more than 5 f

### Example 2: **Get a list of all films with the title, release year, and the name of the actor who acted in the highest number of films.**

In [8]:
nl_query_2 = "Get a list of all films with the title, release year, and the name of the actor who acted in the highest number of films."

# expected
"""
SELECT f.title, f.release_year, a.first_name, a.last_name
FROM film f
JOIN film_actor fa ON f.film_id = fa.film_id
JOIN actor a ON fa.actor_id = a.actor_id
WHERE a.actor_id = (
    SELECT fa2.actor_id
    FROM film_actor fa2
    GROUP BY fa2.actor_id
    ORDER BY COUNT(fa2.film_id) DESC
    LIMIT 1
);
"""

# Generated
"""
SELECT f.title, f.release_year, a.first_name, a.last_name
    FROM film f
    JOIN film_actor fa ON f.film_id = fa.film_id
    JOIN (
      SELECT actor_id, COUNT(film_id) AS film_count
      FROM film_actor
      GROUP BY actor_id
      ORDER BY film_count DESC
      LIMIT 1
    ) AS top_actor ON fa.actor_id = top_actor.actor_id
    JOIN actor a ON top_actor.actor_id = a.actor_id;
"""

response = agent.chat(nl_query_2)

> Running step b49ed934-1767-4c04-951c-19a98f2f8b8e. Step input: Get a list of all films with the title, release year, and the name of the actor who acted in the highest number of films.
Thought: The current language of the user is English. I need to use a tool to help me answer the question. First, I need to check the available tables to understand the database structure.
Action: list_tables
Action Input: {}
Observation: ['actor', 'actor_info', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'nicer_but_slower_film_list', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store']
> Running step f9468981-be45-45d4-9039-1f96ff4c4c92. Step input: None
Thought: To find the films with the title, release year, and the name of the actor who acted in the highest number of films, I need to examine the relevant tables. The likely relevant tables a

### Example 3: **List the films that have been rented by every customer from store 1.**

In [8]:
nl_query_3 = "List the films that have been rented by every customer from store 1."

# Expected
"""
SELECT f.title
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN customer c ON r.customer_id = c.customer_id
JOIN store s ON c.store_id = s.store_id
WHERE s.store_id = 1
GROUP BY f.film_id
HAVING COUNT(DISTINCT c.customer_id) = (
    SELECT COUNT(DISTINCT c2.customer_id)
    FROM customer c2
    WHERE c2.store_id = 1
);
"""

# Generated
"""
SELECT f.title
    FROM film f
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    JOIN customer c ON r.customer_id = c.customer_id
    WHERE c.store_id = 1
    GROUP BY f.film_id
    HAVING COUNT(DISTINCT c.customer_id) = (SELECT COUNT(*) FROM customer WHERE store_id = 1)
  
"""


response = agent.chat(nl_query_3)

> Running step 447b2c97-136d-46ab-a262-31dfa7764268. Step input: List the films that have been rented by every customer from store 1.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step d9c97ffa-8910-49d4-b1c2-dc01985d8ac5. Step input: None
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: list_tables
Action Input: {}
Observation: ['actor', 'actor_info', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'nicer_but_slower_film_list', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store']
> Running step 75fcc5cf-e805-425e-8452-4ca0a3b3c86a. Step input: None
Thought: To find the films rented by every customer from store 1, I need to understand the relationships between the tables, especially those related to re

### Example 4: **Find the customer who rented the most films in each month of 2005. For each month, return the month, the customer's full name, and the total number of films they rented in that month.**

In [15]:
nl_query_4 = "Find the customer who rented the most films in each month of 2005. For each month, return the month, the customer's full name, and the total number of films they rented in that month."

# Expected
"""
WITH MonthlyRentals AS (
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        MONTH(r.rental_date) AS rental_month,
        COUNT(r.rental_id) AS rental_count
    FROM rental r
    JOIN customer c ON r.customer_id = c.customer_id
    WHERE YEAR(r.rental_date) = 2005
    GROUP BY c.customer_id, rental_month
),
RankedRentals AS (
    SELECT
        rental_month,
        customer_id,
        first_name,
        last_name,
        rental_count,
        RANK() OVER (PARTITION BY rental_month ORDER BY rental_count DESC) AS rank
    FROM MonthlyRentals
)
SELECT
    rental_month,
    first_name,
    last_name,
    rental_count
FROM RankedRentals
WHERE rank = 1
ORDER BY rental_month;
"""

# Generated

"""
SELECT 
      month,
      full_name,
      total_rentals
    FROM (
      SELECT 
        DATE_FORMAT(r.rental_date, '%Y-%m') AS month,
        CONCAT(c.first_name, ' ', c.last_name) AS full_name,
        COUNT(r.rental_id) AS total_rentals,
        ROW_NUMBER() OVER (PARTITION BY DATE_FORMAT(r.rental_date, '%Y-%m') ORDER BY COUNT(r.rental_id) DESC) as rn
      FROM 
        rental r
      JOIN 
        customer c ON r.customer_id = c.customer_id
      WHERE 
        YEAR(r.rental_date) = 2005
      GROUP BY 
        month, r.customer_id
    ) as ranked
    WHERE rn = 1
    LIMIT 20
"""
response = agent.chat(nl_query_4)

> Running step 2a2208cf-939c-4249-9b1e-08f7d8917a1c. Step input: Find the customer who rented the most films in each month of 2005. For each month, return the month, the customer's full name, and the total number of films they rented in that month.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step e387c97c-0046-4a98-a9c5-58476c736dce. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 4560edbd-fbb8-4aa1-a542-29c949f04b0d. Step input: None
Thought: The current language of the user is English. I need to use a tool to help me answer the question. First, I need to check the available tables to understand the database structure.
Action: list_tables
Action Input: {}
Observation: ['actor', 'actor_info', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'invento